# 임대주택 단지 내 적정 주차수요 예측 대회

**[배경]**

아파트 단지 내 필요한 주차대수는 ①법정주차대수 ②장래주차수요 중 큰 값에 따라 결정하게되어 있어, 정확한 ②장래주차수요의 산정을 필요로 합니다.<br>
현재 ②장래주차수요는 ‘주차원단위’와 ‘건축연면적’을 기초로하여 산출되고 있으며,<br>
‘주차원단위’는 신규 건축예정 부지 인근의 유사 단지를 피크 시간대 방문하여 주차된 차량대수를 세는 방법으로 조사하고 있습니다.<br>
이 경우 인력조사로 인한 오차발생, 현장조사 시점과 실제 건축시점과의 시간차 등의 문제로 과대 또는 과소 산정의 가능성을 배제할 수 없습니다.<br>

**[주제]**
유형별 임대주택 설계 시 단지 내 적정 주차 수요를 예측

**[평가]**

- 평가산식 : **MAE(Mean Absolute Error)**
- Public 평가 : 전체 Test 데이터 중 무작위 **33% (50단지)**
- Private 평가 : 전체 Test 데이터 중 나머지 **67% (100단지)**

**[외부 데이터 및 사전학습 모델]**

- 공공 데이터와 같이 누구나 얻을 수 있고 법적 제약이 없는 **외부 데이터 허용**
- 사전학습 모델의 경우 **사전학습에 사용된 데이터를 명시**해야함
- 최종 평가시 외부데이터 및 출처 제출

## Data Understanding

**train.csv - 학습용 데이터**

- 단지코드(key)
- 총세대수
- 임대건물구분
- 지역
- 공급유형
- 전용면적
- 전용면적별세대수
- 공가수
- 신분
- 임대료보증금
- 임대료
- 도보 10분거리 내 지하철역 수(환승노선 수 반영)
- 도보 10분거리 내 버스정류장 수
- 단지내주차면수
- 등록차량수(target)

**age_gender_info.csv - 지역 임대주택 나이별, 성별 인구 분포**

- 지역
- 10대미만(여자)
- 10대미만(남자)
- 20대(여자)
- 20대(남자)
- 30대(여자)
- 30대(남자)
- 40대(여자)
- 40대(남자)
- 50대(여자)
- 50대(남자)
- 60대(여자)
- 60대(남자)
- 70대(여자)
- 70대(남자)
- 80대(여자)
- 80대(남자)
- 90대(여자)
- 90대(남자)
- 100대(여자)
- 100대(남자)

In [11]:
PATH = r'C:\Users\Wyatt\wyatt37\data\parking'

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [21]:
age_gender = pd.read_csv(PATH + '\\age_gender_info.csv')
age_gender

,지역,10대미만(여자),10대미만(남자),10대(여자),10대(남자),20대(여자),20대(남자),30대(여자),30대(남자),40대(여자),...,60대(여자),60대(남자),70대(여자),70대(남자),80대(여자),80대(남자),90대(여자),90대(남자),100대(여자),100대(남자)
0,경상북도,0.030158,0.033195,0.056346,0.061360,0.060096,0.067859,0.053433,0.049572,0.083660,...,0.082684,0.063889,0.047717,0.030172,0.029361,0.011211,0.005578,0.001553,0.000234,0.000014
1,경상남도,0.027400,0.026902,0.053257,0.055568,0.064920,0.070618,0.056414,0.057550,0.077092,...,0.087201,0.069562,0.048357,0.033277,0.027361,0.011295,0.004910,0.001086,0.000179,0.000010
2,대전광역시,0.028197,0.029092,0.040490,0.042793,0.060834,0.064247,0.068654,0.066848,0.074667,...,0.088468,0.070261,0.051010,0.037143,0.032455,0.013751,0.006494,0.001740,0.000298,0.000066
3,경기도,0.038030,0.039507,0.052546,0.053990,0.058484,0.059894,0.072331,0.068704,0.083208,...,0.074237,0.058419,0.042422,0.032725,0.025136,0.012354,0.005390,0.001707,0.000290,0.000067
4,전라북도,0.028089,0.029065,0.059685,0.060080,0.066262,0.070322,0.052027,0.046596,0.077005,...,0.076636,0.068042,0.051025,0.035748,0.035049,0.012641,0.007223,0.001898,0.000158,0.000013
5,강원도,0.028888,0.030682,0.051287,0.052712,0.060374,0.063157,0.059821,0.054321,0.076201,...,0.088423,0.070014,0.047732,0.032760,0.033515,0.013027,0.007628,0.001677,0.000319,0.000017
6,광주광역시,0.031994,0.034566,0.049851,0.050254,0.065084,0.066875,0.066888,0.064416,0.080028,...,0.075930,0.059586,0.048552,0.031754,0.029749,0.010341,0.006343,0.000895,0.000353,0.000013
7,충청남도,0.031369,0.031711,0.059077,0.062422,0.067975,0.072622,0.065095,0.067303,0.078860,...,0.070278,0.057692,0.042296,0.028682,0.024514,0.010900,0.005429,0.001549,0.000219,0.000123
8,부산광역시,0.022003,0.022947,0.032681,0.035512,0.053796,0.057233,0.047049,0.048866,0.061952,...,0.109297,0.085294,0.078743,0.053388,0.047908,0.020228,0.008043,0.002240,0.000268,0.000028
9,제주특별자치도,0.034690,0.036695,0.060094,0.063080,0.069135,0.069667,0.050808,0.048026,0.080548,...,0.074248,0.055717,0.047944,0.033054,0.026836,0.011332,0.006832,0.000982,0.000368,0.000082


In [22]:
train = pd.read_csv(PATH + '\\train.csv')
train

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,신분,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,일반,15667000,103680,0.0,3.0,813.0,1015
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,일반,15667000,103680,0.0,3.0,813.0,1015
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,일반,27304000,184330,0.0,3.0,813.0,1015
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,일반,27304000,184330,0.0,3.0,813.0,1015
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,일반,27304000,184330,0.0,3.0,813.0,1015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2947,C2532,239,아파트,강원도,국민임대,49.20,19,7.0,일반,11346000,116090,0.0,1.0,171.0,146
2948,C2532,239,아파트,강원도,국민임대,51.08,34,7.0,일반,14005000,142310,0.0,1.0,171.0,146
2949,C2532,239,아파트,강원도,국민임대,51.73,34,7.0,일반,14005000,142310,0.0,1.0,171.0,146
2950,C2532,239,아파트,강원도,국민임대,51.96,114,7.0,일반,14005000,142310,0.0,1.0,171.0,146


In [23]:
test = pd.read_csv(PATH + '\\test.csv')
test

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,신분,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,철거민,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,일반,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,철거민,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,철거민,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,철거민,43497000,296780,0.0,2.0,683.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,산업단지근로자,-,-,0.0,1.0,467.0
1018,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,철거민,6872000,106400,0.0,2.0,300.0
1019,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,철거민,6872000,106400,0.0,2.0,300.0
1020,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,철거민,13410000,144600,0.0,2.0,300.0


In [24]:
sub = pd.read_csv(PATH + '\\sample_submission.csv')
sub

,code,num
0,C1072,0
1,C1128,0
2,C1456,0
3,C1840,0
4,C1332,0
...,...,...
145,C2456,0
146,C1266,0
147,C2152,0
148,C1267,0


In [25]:
age_gender.shape, train.shape, test.shape, sub.shape

((16, 23), (2952, 15), (1022, 14), (150, 2))

In [30]:
train.sample(20)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,신분,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
2512,C1363,2568,아파트,경기도,국민임대,26.85,88,11.0,철거민,17021000,158060,0.0,6.0,1103.0,2550
2202,C2237,70,아파트,충청남도,국민임대,46.87,8,7.0,일반,13986000,95150,0.0,2.0,50.0,55
2008,C2082,970,아파트,경기도,국민임대,46.93,120,19.0,일반,48844000,332510,0.0,2.0,845.0,822
800,C1875,1003,상가,충청남도,임대상가,31.92,1,5.0,NaN,NaN,NaN,NaN,8.0,192.0,209
1688,C2181,872,아파트,충청북도,국민임대,51.93,117,15.0,일반,16723000,161020,0.0,6.0,733.0,683
2881,C2433,511,아파트,경기도,행복주택,36.57,224,19.0,장기근속자,38080000,165000,0.0,2.0,389.0,130
2416,C1933,1022,아파트,광주광역시,공공임대(10년),66.85,100,8.0,일반,35000000,520000,0.0,3.0,1202.0,1730
672,C1775,711,아파트,제주특별자치도,영구임대,26.34,360,5.0,3차탈락자,4028000,74770,0.0,8.0,131.0,96
1550,C1761,602,아파트,강원도,국민임대,51.93,76,18.0,일반,27465000,232200,0.0,2.0,715.0,730
1477,C2070,898,아파트,경기도,국민임대,46.96,152,6.0,일반,21420000,206410,0.0,3.0,909.0,735
